In [2]:
import geopandas as gpd
import geemap
from outils.initializeGEE import initializeEarthEngine
from indexes.sentinel2IndexCalculator import *

# Autentica e inicializa o Google Earth Engine
initializeEarthEngine()

True

In [3]:
# Load indigenous territories data from parquet file
# Only the first 3 territories are selected for this example
gdf = gpd.read_parquet('data/terrasIndigenasBr.parquet')
gdf = gdf[:3]  # Using only 3 territories for demonstration/testing

# Convert GeoDataFrame to Earth Engine FeatureCollection
# This is necessary to use the geometries in Google Earth Engine
gdf_ee = geemap.geopandas_to_ee(gdf)

# Configuration parameters for satellite imagery analysis
start_date = '2022-01-01'
end_date = '2024-12-31'
batch_size = 10  # Number of features to process in each batch

# Create an instance of Sentinel2IndexCalculator
# This calculator will handle the retrieval and processing of Sentinel-2 imagery
calculator = Sentinel2IndexCalculator(
    start_date, 
    end_date, 
    gdf_ee,  
    id_column='terrai_cod',  # Column containing unique identifiers for each territory
    batch_size=batch_size    # Process features in batches to avoid memory issues
)

# Calculate NDVI (Normalized Difference Vegetation Index) for all territories
# NDVI is a measure of vegetation health and density
print(f"Calculating NDVI for {len(gdf)} territories from {start_date} to {end_date}...")
ndvi_df = calculator.calculate_single_index('ndvi')

# Display summary statistics of the resulting NDVI values
print(f"NDVI calculation complete. Results shape: {ndvi_df.shape}")
print("\nNDVI Summary Statistics:")
print(ndvi_df.describe())

# Save results to CSV (optional)
output_file = f"ndvi_results_{start_date.replace('-', '')}_{end_date.replace('-', '')}.csv"
ndvi_df.to_csv(output_file)
print(f"Results saved to {output_file}")

Calculating NDVI for 3 territories from 2022-01-01 to 2024-12-31...


Processing NDVI: 100%|██████████| 132/132 [06:36<00:00,  3.01s/it]

NDVI calculation complete. Results shape: (3954, 3)

NDVI Summary Statistics:
        terrai_cod         ndvi
count  3954.000000  1293.000000
mean    301.000000     0.281969
std     216.052012     0.221811
min     101.000000    -0.273592
25%     101.000000     0.089480
50%     201.000000     0.271693
75%     601.000000     0.424765
max     601.000000     0.875951
Results saved to ndvi_results_20220101_20241231.csv
